In [92]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint
import ipywidgets as wg
from IPython.display import display

R = 8.3145

k_1 = 8.5e7
k_2 = 7e9
k_3 = 1.5e12

E_A1 = 1.5e5
E_A2 = 1.67e5
E_A3 = 2.05e5

def batch(c,t,T):
    r1 = k_1*np.exp(-E_A1/(R*T))
    r2 = k_2*np.exp(-E_A2/(R*T))
    r3 = k_3*np.exp(-E_A3/(R*T))
    
    dcadt = -r1*c[0]
    dcbdt = r1*c[0] - r2*c[1] - r3*c[1]
    dccdt = r2*c[1]
    dcddt = r3*c[1]
    return [dcadt, dcbdt, dccdt, dcddt]


def bp(T, t_r,p):
    t = np.linspace(0,t_r,100)
    ca = p*1e5/(R*T)
    c_0 = [ca,0,0,0]
    c = odeint(batch, c_0, t, args=(T,))
    
    fig1=plt.figure(figsize=(15, 10), dpi = 100)
    plt.plot(t,c[:,0], label="Reactand")
    plt.plot(t,c[:,1], label="Intermediate")
    plt.plot(t,c[:,2], label="Product")
    plt.plot(t,c[:,3], label="Byproduct")
    plt.ylim(-2,27)
    plt.xlabel("Reaction time [s]")
    plt.ylabel("Concentration of component i [mol $\cdot$ m$^{-3}$]")
    plt.legend(loc="best")
    
    

    
def sens(T_int,n, t_r,p):
    T_L = T_int[0]
    T_U = T_int[1]
    T = np.linspace(T_L,T_U,n)
    t = np.linspace(0,t_r,100)
          
    conv = np.arange(n, dtype=float)
    yieldp = np.arange(n, dtype=float)
    yieldnp = np.arange(n, dtype=float)
    selecp = np.arange(n, dtype=float)
    selecnp = np.arange(n, dtype=float)

            
    for i in range(n):
        ca = p*1e5/(R*T[i])
        c_0 = [ca,0,0,0]
        c = odeint(batch, c_0, t, args=(T[i],))
        conv[i] = (c[0,0]-c[-1,0])/c[0,0]*100
        yieldp[i] = (c[-1,2]-c[0,2])/c[0,0]*100
        yieldnp[i] = (c[-1,3]-c[0,3])/c[0,0]*100
        selecp[i] = yieldp[i]/conv[i]*100
        selecnp[i] = yieldnp[i]/conv[i]*100
    
    fig2=plt.figure(figsize=(15, 10), dpi = 100)
    plt.ylim([-2,102])
    plt.xlabel("Reactor Temperature [K]")
    plt.ylabel("Yield, Selectivity and Conversion [%]")
    plt.plot(T,conv, label="Conversion %")
    plt.plot(T,yieldp, label="Yield %")
    plt.plot(T,selecp, label="Selectivity Product %")
    plt.plot(T,selecnp, label="Selectivity Byproduct %")
    plt.legend(loc="best")
    


In [97]:
s1 = {'description_width': 'initial'}
T_text = wg.FloatText(value=776, step=5, description="Reactor temperature $T_R$ [K]", style=s1)
t_text = wg.FloatText(value = 120, step=10, description="Reaction time $t_R$ [s]", style=s1)
p_text = wg.FloatText(value = 1.6, step=0.1, description="Vessel Pressure $p_R$ [bar]", style=s1)
wg.interact(bp, T = T_text, t_r = t_text, p=p_text)


interactive(children=(FloatText(value=776.0, description='Reactor temperature $T_R$ [K]', step=5.0, style=Desc…

<function __main__.bp(T, t_r, p)>

In [100]:
s1 = {'description_width': 'initial'}
T_range = wg.FloatRangeSlider(value=[750,800], min=600, max=1000, step=5.0, description="Temperature Range [K]", continuous_update= False, style=s1)
dp_text = wg.IntText(value = 20, step=5, description="Number of Datapoints", style=s1)
t_text = wg.FloatText(value = 120, step=10, description="Reaction time $t_R$ [s]", style=s1)
p_text = wg.FloatText(value = 1.6, step=0.1, description="Vessel Pressure $p_R$ [bar]", style=s1)
                                                        
wg.interact(sens, T_int = T_range, n = dp_text, t_r = t_text, p=p_text)

interactive(children=(FloatRangeSlider(value=(750.0, 800.0), continuous_update=False, description='Temperature…

<function __main__.sens(T_int, n, t_r, p)>